In [1]:
import pandas as pd
import numpy as np
import json
import re
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D  # for 3D plotting
import plotly.express as px
from sklearn.manifold import TSNE
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
import plotly.io as pio
import os
from os import getenv
from dotenv import load_dotenv
import requests
import ast

x_chat = chatGPT()
x_gemini = gemini()



load_dotenv("../.env",override=True)
GEO_KEY = getenv("geo_api_key")

df = pd.read_csv("../output/bls_df.csv",index_col=0)
tsne = pd.read_csv("../output/tsne.csv",index_col=0)
df = pd.concat([df,tsne],axis=1)


In [2]:


wage_employment = pd.read_csv("../input/wage_employment2022/national2022.csv")
wage_employment = wage_employment[wage_employment.O_GROUP == "detailed"]
wage_employment = wage_employment.rename({"OCC_CODE": "Detailed Occupation"}, axis=1)
wage_employment = wage_employment.replace("#", np.nan).replace("*", np.nan).replace({",": ""}, regex=True)
wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP","JOBS_1000"]] = wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP","JOBS_1000"]].astype("float")
wage_employment["H_MEDIAN"] = wage_employment["H_MEDIAN"].round(0)
wage_employment = wage_employment[["Detailed Occupation","H_MEDIAN","A_MEDIAN","TOT_EMP","JOBS_1000"]]
wage_employment.columns = ["Detailed Occupation","H_MEDIAN_US","A_MEDIAN_US","TOT_EMP_US","JOBS_1000_US"]


all_tasks = df.groupby("Task").aggregate({"onet_weight":"sum"}).reset_index()
task_statements = pd.read_csv("../input/onet/Task Statements.csv")[["Task","Title"]]
all_tasks = all_tasks.merge(task_statements, on=["Task"], how="outer")
onet_occ = pd.read_csv("../input/onet/Occupation Data.csv")[["O*NET-SOC Code","Title"]]
onet_occ.columns = ["Detailed Occupation","onet_title"]
onet_occ["Detailed Occupation"] = onet_occ["Detailed Occupation"].apply(lambda x: x[:-3])
all_tasks = all_tasks.merge(onet_occ, left_on="Title",right_on="onet_title").drop(columns=["Title"])

all_tasks = all_tasks.replace(np.nan,0)
perc_of_10K = (wage_employment.groupby("Detailed Occupation")["TOT_EMP_US"].first()/df["TOT_EMP_US"].sum() * 10000).reset_index() # num of people for every 10K workers
perc_of_10K = perc_of_10K.rename({"TOT_EMP_US":"Percent of 10K workers"},axis=1)


all_tasks = all_tasks.merge(perc_of_10K, on="Detailed Occupation")
ratios = all_tasks['onet_weight'] / all_tasks['Percent of 10K workers']
all_tasks["automated_weight"] = np.minimum(ratios, 1)


all_occupations = all_tasks.groupby("Detailed Occupation").aggregate({"automated_weight":"sum","Task":"count", "onet_title":"first"})
all_occupations.columns = ["occupation_# tasks automated","occupation_# of tasks","onet_title"]
all_occupations["occupation_onet_rating"] = all_occupations["occupation_# tasks automated"]/all_occupations["occupation_# of tasks"]
df = df.merge(all_tasks[["Task","automated_weight","Percent of 10K workers"]], on="Task")
df = df.merge(all_occupations,on=["Detailed Occupation","onet_title"])



all_occupations = all_occupations.merge(wage_employment, on="Detailed Occupation")
all_tasks = all_tasks.merge(wage_employment, on="Detailed Occupation")
all_tasks = all_tasks.rename({"Title":"onet_title"},axis=1)

In [ ]:
codes = pd.read_csv("../input/soc_codes/soc_codes.csv", index_col=0)
all_occupations = all_occupations.merge(codes,on="Detailed Occupation")
all_tasks = all_tasks.merge(codes,on="Detailed Occupation")




In [ ]:
dwa = pd.read_csv("../input/onet/Tasks to DWAs.csv")[3:].reset_index(drop=True)
dwa = dwa[["DWA ID","DWA Title","Task"]]
# dwa = dwa.rename({"ID":"Task ID"},axis=1)
grouped = dwa.groupby("Task").aggregate({"DWA ID":"count"}).apply(lambda x: 1/x).reset_index()
grouped = grouped.rename({"DWA ID":"dwa_count"},axis=1)
dwa = dwa.merge(grouped,on="Task")
dwa_ref = pd.read_csv("../input/onet/DWA Reference.csv")[["Element Name","DWA ID"]]
dwa_ref = dwa_ref.rename({"Element Name":"activity"},axis=1)

tf = df.merge(dwa, on="Task",how="left")
tf = tf.merge(dwa_ref, on="DWA ID",how="left")
print(len(tf[tf["DWA ID"].isnull()]))

print(len(tf))

In [ ]:
def parse_bls(wage_employment,type):
    wage_employment = wage_employment[wage_employment.O_GROUP == "detailed"]
    wage_employment = wage_employment.rename({"OCC_CODE": "Detailed Occupation"}, axis=1)
    wage_employment = wage_employment.replace("#", np.nan).replace(r"\*+", np.nan, regex=True).replace(",", "", regex=True)
    wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP","JOBS_1000"]] = wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP", "JOBS_1000"]].astype("float")
    wage_employment["H_MEDIAN"] = wage_employment["H_MEDIAN"].round(0)
    wage_employment = wage_employment[["AREA","Detailed Occupation","H_MEDIAN","A_MEDIAN","TOT_EMP","JOBS_1000"]]
    wage_employment.columns = ["AREA","Detailed Occupation","H_MEDIAN_"+type,"A_MEDIAN_"+type,"TOT_EMP_"+type, 'JOBS_1000_'+type]
    return wage_employment


nonmetro = pd.read_csv("../input/wage_employment2022/non-metro.csv")
metro = pd.read_csv("../input/wage_employment2022/metro.csv")
nonmetro = parse_bls(nonmetro,"RURAL")
metro = parse_bls(metro,"METRO")


In [ ]:
#figures out how much to weight each area
def calculate_weights(df,type):
    grouped = df.groupby("Detailed Occupation")["TOT_EMP_"+type].sum().reset_index()
    grouped = grouped.rename({"TOT_EMP_"+type: "sum_"+type}, axis=1)
    df = df.merge(grouped, on="Detailed Occupation")
    df["weight_area_"+type] = df["TOT_EMP_"+type] / df["sum_"+type]
    df["weighted_JOBS_1000_"+type] = df["JOBS_1000_"+type]*df["weight_area_"+type]
    return df
metro = calculate_weights(metro,"METRO")
nonmetro = calculate_weights(nonmetro,"RURAL")

In [ ]:
metro_grouped = metro.groupby("Detailed Occupation").aggregate({"H_MEDIAN_METRO":"mean","TOT_EMP_METRO":"sum","A_MEDIAN_METRO":"mean", "weighted_JOBS_1000_METRO":"sum"},axis=1).reset_index()
nonmetro_grouped = nonmetro.groupby("Detailed Occupation").aggregate({"H_MEDIAN_RURAL":"mean","TOT_EMP_RURAL":"sum","A_MEDIAN_RURAL":"mean","weighted_JOBS_1000_RURAL":"sum"},axis=1).reset_index()

